# PyUCX-AI Multi-Agent Framework

This notebook provides an interactive interface for running the PyUCX-AI framework to analyze Python files and plan Unity Catalog migrations.

## Overview
The framework uses multiple AI agents to:
- Analyze Python files and lint issues
- Create migration plans
- Generate Unity Catalog compatible code
- Validate and report on migrations


## 1. Setup and Imports


In [1]:
#!/usr/bin/env python3
import sys
import os
import logging
from pathlib import Path
from typing import List, Dict, Any, Optional
from datetime import datetime

# Add the src directory to Python path
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath('.')), 'src'))

from src.utils.logging_setup import quick_setup
from src.utils.config_manager import ConfigManager
from src.core.langgraph_framework import (
    PyUCXFramework, 
    load_notebooks_from_folder,
    load_python_files_from_folder, 
    load_lint_data_from_file,
    save_results_to_file
)

print("✅ Imports completed successfully!")


✅ Imports completed successfully!


## 2. Configuration Parameters

Configure the framework parameters. Modify these as needed for your specific use case.


In [2]:
# Configuration Parameters
CONFIG = {
    "input_folder": "data/sample_python_files",  # Path to Python files to analyze
    "lint_file": "data/lint_outputs/sample_python_scenarios.txt",  # UCX lint output file
    "output_file": "migration_analysis_results.json",  # Output file for results
    "config_file": None,  # Optional: Path to custom config file
    "log_level": "INFO",  # Logging level: DEBUG, INFO, WARNING, ERROR, CRITICAL
    "log_file": None,  # Optional: Path to log file (None = console only)
    "max_iterations": 200,  # Maximum workflow iterations
    "thread_id": "notebook-session",  # Thread ID for workflow execution
    "verbose": True  # Enable verbose output
}

print("📋 Configuration set:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")


📋 Configuration set:
  input_folder: data/sample_python_files
  lint_file: data/lint_outputs/sample_python_scenarios.txt
  output_file: migration_analysis_results.json
  config_file: None
  log_level: INFO
  log_file: None
  max_iterations: 200
  thread_id: notebook-session
  verbose: True


## 3. Setup Logging and Configuration


In [3]:
# Setup logging
log_level = "DEBUG" if CONFIG["verbose"] else CONFIG["log_level"]
logger = quick_setup(log_level=log_level, log_file=CONFIG["log_file"])
logger.info("PyUCX-AI Framework starting...")

# Setup configuration
if CONFIG["config_file"]:
    config = ConfigManager(config_path=CONFIG["config_file"])
else:
    config = ConfigManager()

# Override with notebook parameters
config.update({
    "log_level": CONFIG["log_level"],
    "log_file": CONFIG["log_file"],
    "max_iterations": CONFIG["max_iterations"]
})

print("🔧 Logging and configuration setup completed!")


2025-09-16 10:38:40 | INFO     | src.utils.logging_setup | Logging initialized - Level: DEBUG, File: None
2025-09-16 10:38:40 | INFO     | pyucx_ai | PyUCX-AI Framework starting...
2025-09-16 10:38:40 | INFO     | src.utils.config_manager | Loaded environment variables from .env
2025-09-16 10:38:40 | INFO     | src.utils.config_manager | Configuration loaded successfully
2025-09-16 10:38:40 | DEBUG    | src.utils.config_manager | Configuration updated with 3 values


🔧 Logging and configuration setup completed!


## 4. Validate Configuration


In [4]:
# Validate configuration
if not config.validate_config():
    raise Exception("Configuration validation failed. Please check your settings.")
    
print("✅ Configuration validation passed!")


2025-09-16 10:38:44 | INFO     | src.utils.config_manager | Configuration validation passed


✅ Configuration validation passed!


## 5. Validate Input Files


In [5]:
def validate_inputs(input_folder: str, lint_file: str, output_file: str) -> bool:
    """Validate input files and directories."""
    errors = []

    # Check input folder
    input_path = Path(input_folder)
    if not input_path.exists():
        errors.append(f"Input folder does not exist: {input_folder}")
    elif not input_path.is_dir():
        errors.append(f"Input path is not a directory: {input_folder}")
    else:
        # Check for Python files
        python_files = list(input_path.glob("*.py"))
        if not python_files:
            errors.append(f"No Python files found in: {input_folder}")

    # Check lint file
    lint_path = Path(lint_file)
    if not lint_path.exists():
        errors.append(f"Lint file does not exist: {lint_file}")
    elif not lint_path.is_file():
        errors.append(f"Lint path is not a file: {lint_file}")

    # Check output directory
    output_path = Path(output_file)
    try:
        output_path.parent.mkdir(parents=True, exist_ok=True)
    except Exception as e:
        errors.append(f"Cannot create output directory: {e}")

    if errors:
        print("❌ Validation Errors:")
        for error in errors:
            print(f"  - {error}")
        return False

    return True

# Validate inputs
if not validate_inputs(CONFIG["input_folder"], CONFIG["lint_file"], CONFIG["output_file"]):
    raise Exception("Input validation failed!")

print("✅ Input validation passed!")


✅ Input validation passed!
